In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pybie2d
import matplotlib as mpl
import matplotlib.pyplot as plt

star = pybie2d.misc.curve_descriptions.star
GSB = pybie2d.boundaries.global_smooth_boundary.global_smooth_boundary.Global_Smooth_Boundary

def check_GSB_test_inside_points(NBfactor):
    '''
    Test the boundary._test_inside_point method, using  points that are known to be inside/outside a star-shaped closed globally-smooth boundary
    '''
    
    NB0=3*5
    NB=NB0*NBfactor
    boundary = GSB(c=((star(NB,x=1,y=1,r=1,a=0.6,f=7))))
    
    print(f"NB={NB}")
    #two points on the boundary
    pc = boundary.c[[1*NB//5,2*NB//3]]
    eps=1e-10
    t=np.linspace(eps,1-eps,30)
    inner_pts=pc[0]+(pc[1]-pc[0])*t-0.5*np.sin(np.pi*t)*(pc[1].real-pc[0].real)
    outer_pts=np.concatenate((pc[1]+(pc[1]-pc[0])*t+0.5*np.sin(np.pi*t)*(pc[1].real-pc[0].real),
                              pc[0]-(pc[1]-pc[0])*t+0.5*np.sin(np.pi*t)*(pc[1].real-pc[0].real)))


    
    check_inpts=np.array([False]*len(inner_pts))
    for i,inpt in enumerate(inner_pts):
        check_inpts[i]=boundary._test_inside_point(inpt)
    
    if not np.all(check_inpts): 
        print(f'test_inside_point failed for {np.sum((~check_inpts)*1)} /{30} points that are inside the boundary')
    failed_inpts=inner_pts[~check_inpts]
    success_inpts=inner_pts[check_inpts]

   
    check_outpts=np.array([False]*len(outer_pts))    
    
    for i,outpt in enumerate(outer_pts):
        check_outpts[i]=~boundary._test_inside_point(outpt)
    
    if not np.all(check_outpts): print(f'test_inside_point failed for {np.sum((~check_outpts)*1)} /{60} points that are outside the boundary')

    failed_outpts=outer_pts[~check_outpts]
    success_outpts=outer_pts[check_outpts]
    failed_pts=np.concatenate((failed_inpts,failed_outpts))
    success_pts=np.concatenate((success_inpts,success_outpts))
    return boundary.c,failed_pts,success_pts




pts,failed_pts,success_pts=check_GSB_test_inside_points(10)
fig, ax = plt.subplots(1,1,figsize=(12,12))
ax.set_aspect('equal')

ax.plot(pts.real,pts.imag,'k-')
ax.plot(failed_pts.real,failed_pts.imag,'rx')
ax.plot(success_pts.real,success_pts.imag,'bx') 

assert(len(failed_pts)==0)